## A MultiServer Queue

Let's develop a simulation for single FIFO Queue that leads to 3 servers.  
 - Customer arrivals follow a poisson process with rate $\lambda$, (i.e. the interarrival times are exponentially distributed with mean $1/\lambda$).
 - Service Times are follow an exponential distribution with mean $1/\mu$.
 - There are $c$ different servers.

 We want to simulate one replication path with $100$ customers.

In [7]:
import numpy as np  

In [8]:
def sim_one_path(lam, mu, c, numCustomers):
    #This information will help us track the "state" of the simulation
    #create the c different queues and keep track of when the the current customer finishes service
    servers = [np.inf] * c
    customers_in_service = [-1] * c #this will track which customer is being served by each server, -1 means no customer is being served
    queue = [] #the queue starts empty.  It will contain a list of the customers currently waiting.
    next_arrival = np.random.exponential(1/lam)

    #this is where we will log outputs of simulation
    #we will track the arrival time of each customer, when they start service, and when they depart service
    A = np.zeros(numCustomers)
    S = np.zeros(numCustomers)
    D = np.zeros(numCustomers)
    A[0] = next_arrival

    now = 0
    indx_arriving_customer = 0

    while True:
        #fast forward time untl the next "event" happens
        next_service_completion = np.min(servers)
        if next_arrival < next_service_completion:
            now = next_arrival
            
            #if there's an empty server, customer enters service
            if np.max(servers) == np.inf:
                indx_empty_server = np.argmax(servers)
                servers[indx_empty_server] = now + np.random.exponential(1/mu)
                customers_in_service[indx_empty_server] = indx_arriving_customer
                
                #log the start of service
                if indx_arriving_customer < numCustomers:
                    S[indx_arriving_customer] = now

            else: #all servers busy, customer enters queue.
                queue.append(indx_arriving_customer)

            next_arrival = now + np.random.exponential(1/lam)
            indx_arriving_customer += 1

            #only log the arrival if it's in the first num_customers
            if indx_arriving_customer < numCustomers:
                #log the arrival time
                A[indx_arriving_customer] = next_arrival

        else: #the next event is a service completion
            now = next_service_completion
            indx_completing_server = np.argmin(servers)

            #log the departure time if its in teh first numCustomers
            leaving_customer = customers_in_service[indx_completing_server]
            if leaving_customer < numCustomers:
                D[leaving_customer] = now


            #if there are customers waiting, the next customer enters service
            if len(queue) > 0:
                servers[indx_completing_server] = now + np.random.exponential(1/mu)
                next_customer = queue[0] 
                queue = queue[1:] #remove the first customer from the queue
                customers_in_service[indx_completing_server] = next_customer

                #log the start of service for the next customer if it's within first numCustomers
                if next_customer < numCustomers:
                    S[next_customer] = now
                
            else: #no customers waiting, server is now empty
                servers[indx_completing_server] = np.inf
                customers_in_service[indx_completing_server] = -1
            
        #need to check if the first numCustomer arrivals have all been processed
        if np.min(D[:numCustomers]) > 0:
            break
    return A, S, D

In [9]:
A, S, D = sim_one_path(1, .8, 2, 30)

#print an array with columns A, S, D, with entries rounded to two digits
output = np.vstack((A, S, D)).T
print("Arrival, Start, Departure")
print(np.round(output, 2))


Arrival, Start, Departure
[[ 1.15  1.15  1.77]
 [ 1.17  1.17  1.92]
 [ 1.17  1.77  2.67]
 [ 3.97  3.97  4.37]
 [ 6.33  6.33  7.48]
 [ 6.43  6.43  6.91]
 [ 6.81  6.91  7.5 ]
 [ 9.04  9.04 10.  ]
 [ 9.34  9.34 10.47]
 [10.31 10.31 12.89]
 [10.63 10.63 10.77]
 [11.35 11.35 12.9 ]
 [15.87 15.87 16.14]
 [16.14 16.14 18.2 ]
 [19.55 19.55 19.93]
 [19.92 19.92 20.38]
 [20.04 20.04 20.16]
 [23.04 23.04 23.78]
 [24.4  24.4  24.58]
 [25.28 25.28 27.52]
 [26.53 26.53 26.75]
 [26.92 26.92 27.28]
 [27.69 27.69 27.86]
 [27.87 27.87 29.06]
 [28.48 28.48 29.89]
 [28.6  29.06 30.95]
 [28.87 29.89 31.49]
 [28.9  30.95 31.61]
 [31.47 31.49 34.09]
 [32.83 32.83 34.96]]


#### For you:  Write a function that takes in the arrays A, S, D and computes 
 - The average waiting time across customers
 - The average length of the queue (averaged over time)

In [10]:
avg_waiting_time = np.mean(S-A)
print(f"Average waiting time: {avg_waiting_time:.2f}")

def avg_queue_length(numDiscretization_points = 100):
    #create an equispaced time grid between 0 and the last departure
    time_grid = np.linspace(0, np.max(D), numDiscretization_points)
    #calculate teh queue length at each time on the time grid
    queue_length = np.zeros(numDiscretization_points)
    for i, t in enumerate(time_grid):
        #count how many customers are in the queue at time t
        queue_length[i] = np.sum((A <= t) & (D > t))
    #return the average queue length
    return np.mean(queue_length)
avg_length = avg_queue_length()
print(f"Average queue length: {avg_length:.2f}")


Average waiting time: 0.14
Average queue length: 0.96


### (For you): Compute the expected average waiting time across the first 100 arrivals using a simulation with 1000 path

In [11]:
exp_avg_waiting = 0.
for rep in range(1000):
    A, S, D = sim_one_path(1, .8, 2, 30)
    exp_avg_waiting += np.mean(S - A)

exp_avg_waiting /= 1000
print(f"Expected average waiting time: {exp_avg_waiting:.2f}")


Expected average waiting time: 0.60


### More interesting service times
Suppose you believe that 10\% of customers have very difficult requests that take a long time for service, say they take $1/\mu_2 > 1/\mu_1$ time.  How would you alter your simulation?

In [12]:
def gen_service_distribution(mu_fast, mu_slow, p_slow):
    #flip acoin with probability p_slow to determine if the service time is fast or slow
    if np.random.rand() < p_slow:
        return np.random.exponential(1/mu_slow)
    else:
        return np.random.exponential(1/mu_fast)
    
#if you read the follwoing code, it's identical to the other simulation except where we simulate servie times we call above function!
#said differently, we only needed to change a few lines of our simulation to handle potentially slow customers!
def sim_one_path_with_mixed_service(lam, mu_fast, mu_slow, p_slow, c, numCustomers):    
    #This information will help us track the "state" of the simulation
    #create the c different queues and keep track of when the the current customer finishes service
    servers = [np.inf] * c
    customers_in_service = [-1] * c #this will track which customer is being served by each server, -1 means no customer is being served
    queue = [] #the queue starts empty.  It will contain a list of the customers currently waiting.
    next_arrival = np.random.exponential(1/lam)

    #this is where we will log outputs of simulation
    #we will track the arrival time of each customer, when they start service, and when they depart service
    A = np.zeros(numCustomers)
    S = np.zeros(numCustomers)
    D = np.zeros(numCustomers)
    A[0] = next_arrival

    now = 0
    indx_arriving_customer = 0

    while True:
        #fast forward time untl the next "event" happens
        next_service_completion = np.min(servers)
        if next_arrival < next_service_completion:
            now = next_arrival
            
            #if there's an empty server, customer enters service
            if np.max(servers) == np.inf:
                indx_empty_server = np.argmax(servers)
                #servers[indx_empty_server] = now + np.random.exponential(1/mu)  #changed to below
                servers[indx_empty_server] = now + gen_service_distribution(mu_fast, mu_slow, p_slow)
                customers_in_service[indx_empty_server] = indx_arriving_customer
                
                #log the start of service
                if indx_arriving_customer < numCustomers:
                    S[indx_arriving_customer] = now

            else: #all servers busy, customer enters queue.
                queue.append(indx_arriving_customer)

            next_arrival = now + np.random.exponential(1/lam)
            indx_arriving_customer += 1

            #only log the arrival if it's in the first num_customers
            if indx_arriving_customer < numCustomers:
                #log the arrival time
                A[indx_arriving_customer] = next_arrival

        else: #the next event is a service completion
            now = next_service_completion
            indx_completing_server = np.argmin(servers)

            #log the departure time if its in teh first numCustomers
            leaving_customer = customers_in_service[indx_completing_server]
            if leaving_customer < numCustomers:
                D[leaving_customer] = now


            #if there are customers waiting, the next customer enters service
            if len(queue) > 0:
                #servers[indx_completing_server] = now + np.random.exponential(1/mu)  #changed to below
                servers[indx_completing_server] = now + gen_service_distribution(mu_fast, mu_slow, p_slow)
                next_customer = queue[0] 
                queue = queue[1:] #remove the first customer from the queue
                customers_in_service[indx_completing_server] = next_customer

                #log the start of service for the next customer if it's within first numCustomers
                if next_customer < numCustomers:
                    S[next_customer] = now
                
            else: #no customers waiting, server is now empty
                servers[indx_completing_server] = np.inf
                customers_in_service[indx_completing_server] = -1
            
        #need to check if the first numCustomer arrivals have all been processed
        if np.min(D[:numCustomers]) > 0:
            break
    return A, S, D


In [15]:
A, S, D = sim_one_path_with_mixed_service(1, 0.8, 2, 0.2, 2, 50)

#print a matrix with columns A, S, D, with entries rounded to two digits
output = np.vstack((A, S, D)).T
print("Arrival, Start, Departure")
print(np.round(output, 2))

avg_waiting_time = np.mean(S - A)
print(f"Average waiting time with mixed service: {avg_waiting_time:.2f}")

Arrival, Start, Departure
[[ 1.97  1.97  2.7 ]
 [ 2.33  2.33  3.71]
 [ 4.94  4.94  5.1 ]
 [ 6.69  6.69  8.46]
 [ 8.09  8.09  9.34]
 [ 8.32  8.46 12.92]
 [ 8.33  9.34  9.97]
 [ 9.23  9.97 10.1 ]
 [11.56 11.56 12.42]
 [13.12 13.12 16.26]
 [13.67 13.67 13.92]
 [16.35 16.35 17.74]
 [16.74 16.74 17.51]
 [16.88 17.51 18.23]
 [17.01 17.74 17.82]
 [17.98 17.98 18.25]
 [18.04 18.23 20.12]
 [18.2  18.25 20.36]
 [19.19 20.12 21.36]
 [19.45 20.36 22.81]
 [20.68 21.36 21.38]
 [20.9  21.38 21.58]
 [21.86 21.86 25.74]
 [22.58 22.81 23.75]
 [24.06 24.06 25.73]
 [24.82 25.73 27.16]
 [25.17 25.74 25.79]
 [26.73 26.73 30.24]
 [28.43 28.43 29.23]
 [28.45 29.23 29.33]
 [29.61 29.61 30.41]
 [30.73 30.73 33.43]
 [31.35 31.35 31.46]
 [32.52 32.52 32.58]
 [34.27 34.27 36.94]
 [36.44 36.44 45.52]
 [37.54 37.54 37.63]
 [38.01 38.01 38.58]
 [38.73 38.73 41.27]
 [38.74 41.27 41.72]
 [38.92 41.72 42.78]
 [39.82 42.78 44.91]
 [40.82 44.91 45.34]
 [43.91 45.34 45.41]
 [45.51 45.51 48.68]
 [45.86 45.86 46.91]
 [47.19 